# AI System Project Paper
---

**Author**: James Wells \
**JHU ID**: jwells52 \
**Class**: Creating AI Enabled Systems \
**Date**: 08-18-2023

## Introduction
---

<!-- This AI system automates the identification process of humpback whales using Few Shot Learning [1]. 

Identification of individual animals is a manual process conducted by scientists and it is a time consuming because scientist are required to manually annotate hundreds of data points. Additionally, each data point is scrutinized heavily because the discriminative features for humpback whales are subtle, making it difficult to accurately discriminate individuals. Lastly, traditional deep learning methodologies are driven by data hungry algorithms that require an extensive amount of data and training steps to obtain high performing results. In the case of animal identification, available data on individuals is limited and insufficient for fully training a deep learning network.

This AI system attempts to enhance the identification process of humpback whales by using Few Shot Learning, specifically Prototypical Networks. These networks are simple, yet effective algorithm for use-cases that have very limited data. Furthermore, this system is a web application that allows users to upload images of whales for automatic identification. -->


The purpose of this AI system is to automatically identify individual humpback whales from images containing their flukes (i.e tail).

Idenitfication of individual humpback whales is a manual process that is time-consuming and mentally exhaustive. Deep learning can be utilized to automate this process, however there are challenges with collecting a sufficient dataset for training a traditional deep learning algorithm such as Convolutional Neural Networks (CNN) [1].

Fortunately, a new area of AI/ML called Few Shot Learning [2] is tailored for use-cases where limited data is available. Using these new method, a deep learning algorithm can be trained on the limited amount of data available for individual humpback whales and be deployed in a system for automated identification.

The system proposed in this paper is a web application with two components: Web User Interface, REST API for Model Predictions, and a Data Store for storing uploaded images and prediction metadata.

![alt text](images/systems-project-architecture-v4.png "System Architecture") \
*Figure 1: Architecture of Systems Project.*



## Decomposition
---

Identification of individual animals is a manual process that is extremely time-consuming. When images are collected, scientist are required to manually review each image for annotation. Furthermore, accurately identifying a humpback whale in an image is difficult because the discriminative features of these animals are subtle. Another pain point of this process is that it is highly repetitive which can cause mental fatigue and this ultimately leads to a higher rate of incorrect identifications.


This system will enhance this process by providing a tool to scientist that can show the most probable identity of humpback whales when given images. Automating this process is of high value because the time saved by this tool allows scientist to focus on the analysis they are conducting for the humpback whales they are observing and tracking.

## Domain Expertise
---

Since the AI model that is predicting the identity of humpback whales is a Few Shot Learning algorithm, the user needs to provide images to the system in the form of support and query sets. 

The support set is required to contain *M* individual whales and for each individual whale *K* images are required. To minimize the chance of incorrect identifications, the suggested values for *M* and *K* are listed below:
* M should be $<5$
* K should be $>3$

It is important to note that identity of *M* whales are provided in the support set.

Comparatively, the query set contains the images of humpback whales that are in need of identification. The size of the query set can be whatever the user desires. For each image in the query set, the predicted identity is the identity of a whale in the support set that is *most similar*. Reference *Figure 2* for a visualization of a Few Shot Learning model making a prediction on given images.


![alt text](images/fsl.png "Model Inference") \
*Figure 2: Example of model inference when M and K both set to 3.*

## Data
---

## Design
---

## Diagnosis
---

## Deployment
---

## References

[1] Keiron O'Shea, Ryan Nash. "*An introduction to convolutional neural networks*". https://arxiv.org/pdf/1511.08458.pdf.\
[2] Archit Parnami, Minwoo Lee. "Learning From Few Examples: A summary of approaches to Few Shot Learning." 2022. arXiv:2203.04291v1 [cs.LG]